# Compute a temporal aggregation and plot on a map

Import PyOphidia and connect to server instance

In [ ]:
from PyOphidia import cube, client
cube.Cube.setclient(read_env=True)

Import data, compute average over time and extract values

In [ ]:
mycube = cube.Cube.importnc(src_path='/data/tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',
                            measure='tasmin',imp_dim='time',import_metadata='yes',ncores=4)
mycube2 = mycube.reduce(operation='max',ncores=4)
mycube3 = mycube2.rollup(ncores=4)
data = mycube3.export_array()

Plot output data on map

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fig = plt.figure(figsize=(15, 6), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

lat = data['dimension'][0]['values'][ : ]
lon = data['dimension'][1]['values'][ : ]
var = data['measure'][0]['values'][ : ]
var = np.reshape(var, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(var)-np.nanmin(var))/20
clevs = np.arange(np.nanmin(var),np.nanmax(var)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.jet)
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)

plt.title('Minimum Near-Surface Air Temperature (deg K)')
plt.show()

Export result to NetCDF file

In [ ]:
mycube3.exportnc2(output_path='/home/ophidia',export_metadata='yes')

A file called *tasmin.nc* will be created in your home folder

Clear your workspace before running other notebooks

In [ ]:
cube.Cube.deletecontainer(container='tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',force='yes')